## Imports

In [1]:
import pandas as pd
import configparser
import os

In [2]:
cwd = os.getcwd()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Config

In [3]:
#Creating configuration
config_path = os.path.join(cwd, '../docs/config.ini')
config = configparser.ConfigParser()
config.read(config_path)

['c:\\Users\\tasco\\OneDrive\\Python\\Projects\\financialstatements\\financialstatements\\notebooks\\../docs/config.ini']

## Reading Data

### Credit Card Data

In [4]:
# Creating directory to credit card data
CREDITCARD_DIRECTORY = os.path.join(cwd,config.get("data_inputs_directory", "CREDITCARD_DIRECTORY"))

In [5]:
# Listing out files in credit card directory
creditcard_files = os.listdir(CREDITCARD_DIRECTORY)

In [6]:
# Creating paths to data
creditcard_paths = []
for file in creditcard_files:
    path = os.path.join(CREDITCARD_DIRECTORY, file)
    creditcard_paths.append(path)

In [7]:
# Creating data frames
dfs = []
for path in creditcard_paths:
    df = pd.read_csv(path)
    dfs.append(df)

In [8]:
# appending data together
creditcard_df = pd.concat(dfs).reset_index(drop=True)

### Table Data

In [9]:
# Reading 
COA_DATA = config.get('table_files', 'COA_DATA')
MONTH_DATA = config.get('table_files', 'MONTH_DATA')

In [10]:
coa_mcc_df = pd.read_excel(COA_DATA, sheet_name='coa_mcc_link_table')

In [11]:
month_df = pd.read_excel(MONTH_DATA)

## ETL

In [12]:
# Changing Date to Datetime dtype
creditcard_df['Date'] = pd.to_datetime(creditcard_df['Date'])

In [13]:
# Changing Amount to absolute amount
creditcard_df['Amount'] = creditcard_df['Amount'].abs()

In [14]:
creditcard_df = creditcard_df.pivot_table(index = ['Date', 'Name', 'Memo'], columns='Transaction', values='Amount')
creditcard_df = creditcard_df.reset_index()

In [15]:
# Delimiting memo field
creditcard_df[['Memo', 'MCC', 'Blank']] = creditcard_df['Memo'].str.split(';', expand=True)

In [16]:
creditcard_df['MCC'] = creditcard_df['MCC'].str[-4:]

In [17]:
creditcard_df['MCC'] = creditcard_df['MCC'].astype('int')

In [18]:
creditcard_df = pd.merge(creditcard_df, 
                        coa_mcc_df, 
                        how='left',
                        on='MCC')

In [19]:
# Creating function for counting number of nulls
def missing_gl_check(df):
    null_count = df['GL_Code'].isnull().sum()
    return null_count

In [20]:
# Creating function for identifying rows with null GL_Codes
def missing_mcc(df):
    df = df[df['GL_Code'].isnull()]
    return print(df)

In [21]:
# Logic gate for checking nulls in the GL_Code column
if missing_gl_check(creditcard_df) != 0:
    print("GL_Codes are missing, need to update the coa and mcc link table")
    missing_mcc(creditcard_df)
else:
    print("No GL_Codes missing")

No GL_Codes missing


In [22]:
creditcard_df['Description'] = creditcard_df['MCC_Description'] + ': ' + creditcard_df['Name']

In [23]:
creditcard_df = creditcard_df[['Date', 'GL_Code', 'Account', 'Description','DEBIT', 'CREDIT']]

In [24]:
creditcard_df = creditcard_df.round(2)

In [25]:
credit_entries = creditcard_df[creditcard_df['CREDIT'].isnull()].reset_index(drop=True).copy()

In [26]:
credit_entries['GL_Code'] = 200101
credit_entries['Account'] = 'EdwardJones MasterCard'
credit_entries = credit_entries.rename(columns={'DEBIT':'CREDIT', 'CREDIT': 'DEBIT'})

In [27]:
credit_entries = credit_entries[['Date', 'GL_Code', 'Account', 'Description','DEBIT','CREDIT']]

In [28]:
debit_entries = creditcard_df[creditcard_df['DEBIT'].isnull()].reset_index(drop=True).copy()

In [29]:
debit_entries['GL_Code'] = 200101
debit_entries['Account'] = 'EdwardJones MasterCard'
debit_entries = debit_entries.rename(columns={'DEBIT':'CREDIT', 'CREDIT': 'DEBIT'})

In [30]:
debit_entries = debit_entries[['Date', 'GL_Code', 'Account', 'Description','DEBIT','CREDIT']]

In [31]:
creditcard_df = pd.concat([creditcard_df, credit_entries, debit_entries]).reset_index(drop=True)

In [32]:
def debit_credit_check(df):
    debit_total = df['DEBIT'].sum()
    credit_total = df['CREDIT'].sum()
    return debit_total == credit_total

In [33]:
if debit_credit_check(creditcard_df) == True:
    print("Debits equal credits. Proceed to next phase")
else:
    print("Something went wrong")

Debits equal credits. Proceed to next phase


In [34]:
creditcard_df['Month_Num'] = creditcard_df['Date'].dt.month

In [35]:
creditcard_df = creditcard_df.sort_values(by='Date').reset_index(drop=True)

In [36]:
creditcard_df['Transaction_ID'] = 'cc-' + creditcard_df['Month_Num'].astype("str") + '-' + (creditcard_df.index + 1).astype("str")

In [37]:
creditcard_df = pd.merge(creditcard_df,
                         month_df,
                         on='Month_Num',
                         how='left')

In [38]:
months = creditcard_df['Month'].unique()

In [39]:
file_path = config.get("data_outputs_directory", "JOURNAL_ENTRIES")
month_num = 1
for month in months:
    df = creditcard_df[creditcard_df['Month'] == month]
    df =  df[['Transaction_ID','Date', 'GL_Code', 'Account', 'Description', 'DEBIT', 'CREDIT']]
    file_name = f'{month_num}_creditcard_{month}_entries.csv'
    df.to_csv(os.path.join(file_path, file_name), index=False)
    month_num += 1